In [1]:
import mysql.connector as sql
import pandas as pd

mydb = sql.connect (
    host = "localhost",
    user = "root",
    password = "ZAQs8271911c$",
    auth_plugin='mysql_native_password',
    buffered = True,          # using this for reading table, panda dont need it
    #multi=True,                 # if wanna use mulitple query
    database = "sql_fr_ai"
)

# auth_plugin is needed, can check by commends 
# SELECT user,authentication_string,plugin,host FROM mysql.user;
# check the columns plugin and user


#? only execute sql query wo passing data to panda dataframe
# mycoursor = mydb.cursor()
# mycoursor.execute(
#     """
#     #! create database
#     create database if not exists database_name;
#     """
# )


#? execute sql query  AND  passing data to panda dataframe
# edit sql query here
sql_query = """
select * 
from  order_report_1
"""
# run sql queries and Importing data from a MySQL database 
# into Pandas data frame

df = pd.read_sql_query(
    sql_query,
    mydb)


df.head()


/home/bruce/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,order_id,date_ymd,taste,amount
0,2016-04-05-0083,2016-04-05,T1,800.0
1,2016-04-05-0028,2016-04-05,T1,800.0
2,2016-04-05-0049,2016-04-05,T1,800.0
3,2016-04-05-0035,2016-04-05,T2,850.0
4,2016-04-05-0016,2016-04-05,T2,850.0


# subquery

In [2]:
sql_query = """
SELECT *
FROM (
    SELECT *
    FROM order_report_1
    UNION ALL
        SELECT *
        FROM  order_report_2
) AS order_report;
"""


df = pd.read_sql_query(
    sql_query,
    mydb)


df


/home/bruce/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,order_id,date_ymd,taste,amount
0,2016-04-05-0083,2016-04-05,T1,800.0
1,2016-04-05-0028,2016-04-05,T1,800.0
2,2016-04-05-0049,2016-04-05,T1,800.0
3,2016-04-05-0035,2016-04-05,T2,850.0
4,2016-04-05-0016,2016-04-05,T2,850.0
...,...,...,...,...
102750,2016-12-16-0039,2016-12-16,T7,850.0
102751,2016-12-16-0006,2016-12-16,T7,850.0
102752,2016-12-16-0052,2016-12-16,T7,850.0
102753,2016-12-16-0022,2016-12-16,T7,850.0


In [6]:
sql_query = """
SELECT order_report.taste, sum(amount)
FROM (
    SELECT *
    FROM order_report_1
    UNION ALL
        SELECT *
        FROM  order_report_2
) AS order_report
GROUP BY taste;
"""


df = pd.read_sql_query(
    sql_query,
    mydb)


df


/home/bruce/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,taste,sum(amount)
0,T1,30436000.0
1,T2,20533450.0
2,T3,12499200.0
3,T4,7220250.0
4,T5,6289000.0
5,T6,3816000.0
6,T7,4057050.0


# parctice from academic ai

In [1]:
import mysql.connector as sql
import pandas as pd

mydb = sql.connect (
    host = "localhost",
    user = "root",
    password = "ZAQs8271911c$",
    auth_plugin='mysql_native_password',
    buffered = True,          # using this for reading table, panda dont need it
    #multi=True,                 # if wanna use mulitple query
    database = "sql_fr_ai"
)

# LEFT JOIN daily_amount and weather
sql_query = """
CREATE VIEW datamart AS
SELECT 
    da.*,
    we.avetemp,
    we.totprecip
FROM 
    daily_amount as da
LEFT JOIN 
    weather as we
ON 
    da.date_ymd = we.date_ymd
;
"""

df = pd.read_sql_query(
    sql_query,
    mydb)


df


# table daily_amount 
# date_ymd	amount_day

# table  weather
# date_ymd	weekday	avetemp	totprecip

/home/bruce/.local/lib/python3.10/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


TypeError: 'NoneType' object is not iterable

In [ ]:
CREATE VIEW datamart_grouped_classified as
SELECT
	amount_day,
	# for column 気温区分
	CONCAT # conbine 3 strings
	(
		CAST( # change int to string
					 (FLOOR( avetemp / 5) * 5 ) AS CHAR
				   ) ,
		'-',
		CONVERT(  # change int to string
							( ( FLOOR(avetemp / 5) + 1 ) * 5), CHAR
							)    
	) AS avetemp_5,
	# for column holiday
	CASE
				WHEN weekday = "Sat" OR weekday = "Sun" OR public_holiday <> Null
							THEN "休日"
							ELSE "平日" 
	END AS holiday,
	# for column 雨   (homework was asking about change it to flag 0 , 1)
	CASE
			WHEN totprecip > 0
						THEN "降った"
						ELSE "降ってない"
	END AS 雨
FROM
	datamart;